In [3]:
import torch
import numpy as np
import torch.nn as nn
from sklearn.metrics import f1_score, recall_score, precision_score
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import sklearn
import json
from scipy.io import mmread, mmwrite, mminfo
import os
import collections
import dgl
pd.__version__, dgl.__version__

('0.25.1', '0.9.1')

In [6]:
from dgl import DGLHeteroGraph

得选 anaconda3 (Python 3.7.3) 那个环境，不然 dgl.DGLHeteroGraph 会出错

In [2]:
corp_data = torch.load('../data/ChinaCorp_1x_2023-06-07.pt')
corp_data.keys()

dict_keys(['Stkcd_year', 'label', 'split_idx', 'fin_ratio', 'nonfin_ratio', 'mda', 'fin_seq', 'nonfin_seq', 'mda_seq', 'node_seq', 'seq_len', 'graphs'])

In [6]:
fraud_index, nonfraud_index = [], []
for i in range(len(corp_data['Stkcd_year'].numpy())):
    id, year = corp_data['Stkcd_year'][i].numpy()
    if corp_data['label'][i].item() == 1:
        fraud_index.append((id, year))
    else:
        nonfraud_index.append((id, year))
        # index = [(i,j) for i,j in corp_data['Stkcd_year'].numpy().tolist()]
print(len(fraud_index), len(nonfraud_index))

937 937


#### Appendix Table A1: Descriptive statistics of fraud dataset

In [5]:
finRatio = pd.read_excel('../data/会计信息质量-财务指标/AIQ_LCFinIndexY_s1.xlsx',encoding='gb18030')
nonfin_data = pd.read_csv('../data/非财务指标/nonfin_data_raw.csv')
finRatio = finRatio.set_index(['Symbol', 'EndDate'])
nonfin_data = nonfin_data.set_index(['Symbol','year'])

统计财务指标

In [7]:
ratios_stat = pd.DataFrame(columns=['Variable', 'fr_n', 'fr_mean', 'fr_stdev', 'fr_min', 'fr_median', 'fr_max',
                                    'nfr_n', 'nfr_mean', 'nfr_stdev', 'nfr_min', 'nfr_median', 'nfr_max'], dtype=object)
ratios_stat = ratios_stat.set_index('Variable')
for ratio_name in finRatio.columns[4:]:
    # 欺诈、非欺诈样本 总共有多少行 firm-year 是非空的
    fr_non_empty = finRatio.loc[fraud_index, ratio_name].count()
    nfr_non_empty = finRatio.loc[nonfraud_index, ratio_name].count()
    fr_ratio = finRatio.loc[fraud_index, ratio_name].values
    nfr_ratio = finRatio.loc[nonfraud_index, ratio_name]
    print(f'{ratio_name} 字段共有 {fr_non_empty} 个非空欺诈样本, Mean:{np.nanmean(fr_ratio):.4f}, Max: {np.nanmax(fr_ratio):.4f}, Min: {np.nanmin(fr_ratio):.4f}, Median: {np.nanmedian(fr_ratio):.4f}, StDev: {np.nanstd(fr_ratio):.4f}')
    print(f'{ratio_name} 字段共有 {nfr_non_empty} 个非空非欺诈样本, Mean:{np.nanmean(nfr_ratio):.4f}, Max: {np.nanmax(nfr_ratio):.4f}, Min: {np.nanmin(nfr_ratio):.4f}, Median: {np.nanmedian(nfr_ratio):.4f}, StDev: {np.nanstd(nfr_ratio):.4f}')
    ratios_stat.loc[ratio_name,:] = {
        'fr_n' : fr_non_empty,
        'fr_mean' : round(np.nanmean(fr_ratio), 4), 
        'fr_stdev': round(np.nanstd(fr_ratio),4), 
        'fr_min' : round(np.nanmin(fr_ratio), 4),
        'fr_median': round(np.nanmedian(fr_ratio),4),
        'fr_max': round(np.nanmax(fr_ratio),4),
        'nfr_n': nfr_non_empty,
        'nfr_mean' : round(np.nanmean(nfr_ratio), 4), 
        'nfr_stdev': round(np.nanstd(nfr_ratio),4), 
        'nfr_min' : round(np.nanmin(nfr_ratio), 4),
        'nfr_median': round(np.nanmedian(nfr_ratio),4),
        'nfr_max': round(np.nanmax(nfr_ratio),4)
    }

AQI 字段共有 937 个非空欺诈样本, Mean:1.2664, Max: 80.7903, Min: 0.0588, Median: 1.0106, StDev: 2.9665
AQI 字段共有 937 个非空非欺诈样本, Mean:1.1254, Max: 6.3590, Min: 0.1437, Median: 1.0327, StDev: 0.4955
AT 字段共有 937 个非空欺诈样本, Mean:0.6324, Max: 8.9242, Min: 0.0006, Median: 0.4445, StDev: 0.8181
AT 字段共有 937 个非空非欺诈样本, Mean:0.6484, Max: 7.2154, Min: 0.0040, Median: 0.5312, StDev: 0.5410
CFED 字段共有 937 个非空欺诈样本, Mean:302014129.8750, Max: 17919485000.0000, Min: -8690102854.5800, Median: 70528838.3400, StDev: 1405566286.0198
CFED 字段共有 937 个非空非欺诈样本, Mean:168339407.8912, Max: 10975864563.4900, Min: -4721910852.5400, Median: 33581891.1900, StDev: 972728806.2263
DSIR 字段共有 932 个非空欺诈样本, Mean:1.5955, Max: 199.3061, Min: 0.0000, Median: 0.9961, StDev: 7.3471
DSIR 字段共有 933 个非空非欺诈样本, Mean:1.2104, Max: 47.9970, Min: 0.0000, Median: 1.0087, StDev: 1.8330
DEPI 字段共有 935 个非空欺诈样本, Mean:1.4969, Max: 217.2511, Min: -0.0133, Median: 0.9621, StDev: 7.5413
DEPI 字段共有 936 个非空非欺诈样本, Mean:0.2649, Max: 53.7573, Min: -728.8398, Median: 0.960

统计非财务指标

In [8]:
for ratio_name in finRatio.columns[4:]:
    # 欺诈、非欺诈样本 总共有多少行 firm-year 是非空的
    fr_non_empty = finRatio.loc[fraud_index, ratio_name].count()
    nfr_non_empty = finRatio.loc[nonfraud_index, ratio_name].count()
    fr_ratio = finRatio.loc[fraud_index, ratio_name].values
    nfr_ratio = finRatio.loc[nonfraud_index, ratio_name].values
    print(f'{ratio_name} 字段共有 {fr_non_empty} 个非空欺诈样本, Mean:{np.nanmean(fr_ratio):.4f}, Max: {np.nanmax(fr_ratio):.4f}, Min: {np.nanmin(fr_ratio):.4f}, Median: {np.nanmedian(fr_ratio):.4f}, StDev: {np.nanstd(fr_ratio):.4f}')
    print(f'{ratio_name} 字段共有 {nfr_non_empty} 个非空非欺诈样本, Mean:{np.nanmean(nfr_ratio):.4f}, Max: {np.nanmax(nfr_ratio):.4f}, Min: {np.nanmin(nfr_ratio):.4f}, Median: {np.nanmedian(nfr_ratio):.4f}, StDev: {np.nanstd(nfr_ratio):.4f}')
    ratios_stat.loc[ratio_name,:] = {
        'fr_n' : fr_non_empty,
        'fr_mean' : round(np.nanmean(fr_ratio), 4), 
        'fr_stdev': round(np.nanstd(fr_ratio),4), 
        'fr_min' : round(np.nanmin(fr_ratio), 4),
        'fr_median': round(np.nanmedian(fr_ratio),4),
        'fr_max': round(np.nanmax(fr_ratio),4),
        'nfr_n': nfr_non_empty,
        'nfr_mean' : round(np.nanmean(nfr_ratio), 4), 
        'nfr_stdev': round(np.nanstd(nfr_ratio),4), 
        'nfr_min' : round(np.nanmin(nfr_ratio), 4),
        'nfr_median': round(np.nanmedian(nfr_ratio),4),
        'nfr_max': round(np.nanmax(nfr_ratio),4)
    }

AQI 字段共有 937 个非空欺诈样本, Mean:1.2664, Max: 80.7903, Min: 0.0588, Median: 1.0106, StDev: 2.9665
AQI 字段共有 937 个非空非欺诈样本, Mean:1.1254, Max: 6.3590, Min: 0.1437, Median: 1.0327, StDev: 0.4955
AT 字段共有 937 个非空欺诈样本, Mean:0.6324, Max: 8.9242, Min: 0.0006, Median: 0.4445, StDev: 0.8181
AT 字段共有 937 个非空非欺诈样本, Mean:0.6484, Max: 7.2154, Min: 0.0040, Median: 0.5312, StDev: 0.5410
CFED 字段共有 937 个非空欺诈样本, Mean:302014129.8750, Max: 17919485000.0000, Min: -8690102854.5800, Median: 70528838.3400, StDev: 1405566286.0198
CFED 字段共有 937 个非空非欺诈样本, Mean:168339407.8912, Max: 10975864563.4900, Min: -4721910852.5400, Median: 33581891.1900, StDev: 972728806.2263
DSIR 字段共有 932 个非空欺诈样本, Mean:1.5955, Max: 199.3061, Min: 0.0000, Median: 0.9961, StDev: 7.3471
DSIR 字段共有 933 个非空非欺诈样本, Mean:1.2104, Max: 47.9970, Min: 0.0000, Median: 1.0087, StDev: 1.8330
DEPI 字段共有 935 个非空欺诈样本, Mean:1.4969, Max: 217.2511, Min: -0.0133, Median: 0.9621, StDev: 7.5413
DEPI 字段共有 936 个非空非欺诈样本, Mean:0.2649, Max: 53.7573, Min: -728.8398, Median: 0.960

In [9]:
for ratio_name in nonfin_data.columns:
    fr_non_empty = nonfin_data.loc[fraud_index, ratio_name].count()
    nfr_non_empty = nonfin_data.loc[nonfraud_index, ratio_name].count()
    fr_var = nonfin_data.loc[fraud_index, ratio_name].values
    nfr_var = nonfin_data.loc[nonfraud_index, ratio_name].values
    
    ratios_stat.loc[ratio_name,:] = {
        'fr_n' : fr_non_empty,
        'fr_mean' : round(np.nanmean(fr_var), 4), 
        'fr_stdev': round(np.nanstd(fr_var),4), 
        'fr_min' : round(np.nanmin(fr_var), 4),
        'fr_median': round(np.nanmedian(fr_var),4),
        'fr_max': round(np.nanmax(fr_var),4),
        'nfr_n': nfr_non_empty,
        'nfr_mean' : round(np.nanmean(nfr_var), 4), 
        'nfr_stdev': round(np.nanstd(nfr_var),4), 
        'nfr_min' : round(np.nanmin(nfr_var), 4),
        'nfr_median': round(np.nanmedian(nfr_var),4),
        'nfr_max': round(np.nanmax(nfr_var),4)
    }

In [82]:
ratios_stat.to_csv('../data/variable_stat.csv', encoding='utf-8')

#### Table A2: Frequency of the fraud companies by industry
总共5种大类行业：工业()、综合、房地产、公用事业、商业

In [10]:
train_test = pd.read_csv('../data/train_test_x1.csv', encoding='gb18030')
train_test

,Stkcd,pred_year,time_series,label,industry,asset
0,4,2013,"[2013, 2012, 2011, 2010, 2009, 2008]",1,工业,2.421521e+08
1,633,2013,"[2013, 2012, 2011, 2010, 2009, 2008]",0,工业,2.952462e+08
2,4,2012,"[2012, 2011, 2010, 2009, 2008, 2007]",1,工业,1.929136e+08
3,2058,2012,"[2012, 2011, 2010, 2009, 2008, 2007]",0,工业,2.404752e+08
4,4,2011,"[2011, 2010, 2009, 2008, 2007, 2006]",1,工业,1.963072e+08
...,...,...,...,...,...,...
1869,597,2021,"[2021, 2020, 2019, 2018, 2017, 2016]",0,工业,1.317952e+10
1870,603766,2020,"[2020, 2019, 2018, 2017, 2016, 2015]",1,工业,1.312644e+10
1871,601339,2020,"[2020, 2019, 2018, 2017, 2016, 2015]",0,工业,1.315943e+10
1872,603991,2020,"[2020, 2019]",1,工业,5.717125e+08


In [11]:
fraud_sample = train_test[train_test['label']==1]
fraud_sample = fraud_sample.set_index(['industry'])
ind_distri = {}
for ind in fraud_sample.index:
    ind_distri[ind] = len(set(fraud_sample.loc[ind,'Stkcd'].tolist()))


n_fr_firms = np.sum(list(ind_distri.values()))
print(ind_distri)
for key in ind_distri:
    ind_distri[key] = ind_distri[key]/n_fr_firms
print(ind_distri)

{'工业': 258, '综合': 18, '房地产': 24, '公用事业': 69, '商业': 27}
{'工业': 0.6515151515151515, '综合': 0.045454545454545456, '房地产': 0.06060606060606061, '公用事业': 0.17424242424242425, '商业': 0.06818181818181818}


#### Table A3: Frequency of the fraud companies by violation type

In [12]:
fraud_info = pd.read_csv('../data/frau_time_industry_money_since2003.csv', encoding='gb18030')
fraud_info = fraud_info.set_index(['Stkcd','fraud_year'])
fraud_info

inc_name industry                                  type  \
Stkcd  fraud_year                                                           
4      2010         *ST 国农       工业           ["P2503", "P2515", "P2599"]   
       2011           国农科技       工业           ["P2503", "P2515", "P2599"]   
       2012           国农科技       工业           ["P2503", "P2515", "P2599"]   
       2013           国农科技       工业           ["P2503", "P2515", "P2599"]   
9      2016           中国宝安       综合                    ["P2506", "P2515"]   
...                    ...      ...                                   ...   
688555 2020           泽达易盛     公用事业           ["P2501", "P2502", "P2503"]   
       2021           泽达易盛     公用事业           ["P2501", "P2502", "P2503"]   
688559 2021            海目星       工业           ["P2503", "P2515", "P2599"]   
688586 2020           江航装备       工业  ["P2501", "P2503", "P2504", "P2599"]   
       2021           江航装备       工业  ["P2501", "P2503", "P2504", "P2599"]   

                          asset  restate  
Stkcd  fraud_year                         
4      2010        1.842194e+08    False  
       2011        1.963072e+08    False  
       2012        1.929136e+08    False  
       2013        2.421521e+08    False  
9      2016        2.162293e+10    False  
...                         ...      ...  
688555 2020        9.465675e+08    False  
       2021        1.175543e+09    False  
688559 2021        5.591708e+09    False  
688586 2020        2.987056e+09    False  
       2021        3.485059e+09    False  

[2049 rows x 5 columns]

In [111]:
used_fraud_info = fraud_info.loc[fraud_index,:]
vio_type_distri = collections.defaultdict(set)
for stkd, year in fraud_index:
    for t in json.loads(used_fraud_info.loc[(stkd, year), 'type']):
        if t > 'P2506' or t < 'P2501':
            continue
        vio_type_distri[t].add(stkd)

In [120]:
n_total_corp = np.sum([len(v) for v in vio_type_distri.values()])
for key in vio_type_distri:
    # n_total_corp += len(vio_type_distri[key])
    print(key, len(vio_type_distri[key])/n_total_corp)

P2503 0.2960609911054638
P2506 0.04701397712833545
P2504 0.20330368487928843
P2505 0.24650571791613723
P2501 0.17534942820838628
P2502 0.03176620076238882


787

#### Table A4 Frequency of the fraud companies by fraud duration

In [128]:
id_year = {}
for Stkcd, year in fraud_index:
    if Stkcd in id_year:
        id_year[Stkcd] += 1
    else:
        id_year[Stkcd] = 1

year_id = collections.defaultdict(int)
for id, count in id_year.items():
    year_id[count]+= (1/len(id_year))
year_id

defaultdict(int,
            {4: 0.1226666666666666,
             1: 0.27200000000000035,
             2: 0.28266666666666695,
             5: 0.10133333333333329,
             3: 0.2213333333333337})